In [21]:
from pathlib import Path

import torch

from models.common import DetectMultiBackend
from utils.torch_utils import select_device
from utils.general import (check_img_size, scale_coords, non_max_suppression, cv2)
from utils.segment.general import process_mask

from utils.augmentations import letterbox
import numpy as np

weights=Path().resolve() / 'best.pt'
data=Path(r"D:\QuickSegData\Nuclei2") / 'coco128.yaml'  # dataset.yaml path
source='D:/QuickSegData/Nuclei2/test/images/0a849e0eb15faa8a6d7329c3dd66aabe9a294cccb52ed30a90c8ca99092ae732.png'  # file/dir/URL/glob, 0 for webcam
device='cpu'  # cuda device, i.e. 0 or 0,1,2,3 or cpu
dnn=False  # use OpenCV DNN for ONNX inference
half=False  # use FP16 half-precision inference
imgsz=(640, 640)

In [22]:
# Load model
device = select_device(device)
model = DetectMultiBackend(weights, device=device, dnn=dnn, data=data, fp16=half)
stride, names, pt = model.stride, model.names, model.pt
imgsz = check_img_size(imgsz, s=stride)  # check image size

YOLOv5  2023-5-2 Python-3.11.3 torch-2.0.1 CPU

Fusing layers... 
Model summary: 224 layers, 7398422 parameters, 0 gradients, 25.7 GFLOPs


In [23]:
conf_thres=0.25  # confidence threshold
iou_thres=0.45  # NMS IOU threshold
max_det=1000  # maximum detections per image

In [4]:
# Run inference
bs = 1  # batch_size
model.warmup(imgsz=(1 if pt else bs, 3, *imgsz))  # warmup

In [10]:
# path, im, im0s, vid_cap, s = next(iter(dataset)) # assume only one item
im0 = cv2.imread(source)
im = letterbox(im0, imgsz, stride=stride, auto=True)[0]  # padded resize
im = im.transpose((2, 0, 1))[::-1]  # HWC to CHW, BGR to RGB
im = np.ascontiguousarray(im)  # contiguous

In [24]:
im0.shape

(256, 256, 3)

In [13]:
im = torch.from_numpy(im).to(device)
im = im.half() if model.fp16 else im.float()  # uint8 to fp16/32
im /= 255  # 0 - 255 to 0.0 - 1.0
if len(im.shape) == 3:
    im = im[None]  # expand for batch dim

pred, out = model(im, augment=False, visualize=False)
proto = out[1]

classes = None # filter by class: --class 0, or --class 0 2 3 
agnostic_nms=False,  # class-agnostic NMS
pred = non_max_suppression(pred, conf_thres, iou_thres, classes, agnostic_nms, max_det=max_det, nm=32)

In [17]:
det = pred[0]
i = 0
# p, im0, frame = path, im0s.copy(), getattr(dataset, 'frame', 0)

# if len(det):
masks = process_mask(proto[i], det[:, 6:], det[:, :4], im.shape[2:], upsample=True)  # HWC

# # Rescale boxes from img_size to im0 size
# det[:, :4] = scale_coords(im.shape[2:], det[:, :4], im0.shape).round()

# # Print results
# for c in det[:, 5].unique():
#     n = (det[:, 5] == c).sum()  # detections per class
#     s += f"{n} {names[int(c)]}{'s' * (n > 1)}, "  # add to string